In [6]:
import os
import pandas as pd
import logging
import io
import requests
import threading
import queue
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
from lxml import html
import aiohttp
import asyncio

DATA_PATH = "/purchase/data"

tqdm.pandas()

In [7]:
df = pd.read_excel(os.path.join(DATA_PATH, 'INN_OUT_31_08_2021.xlsx'))
# Data preparing
df['purch'] = df['Реестровый номер закупки'].progress_apply(lambda x:x[1:])
df = df.loc[df['Закупки по']=='44-ФЗ']
df = df.loc[df['Этап закупки']=='Определение поставщика завершено']

100%|██████████| 1488/1488 [00:00<00:00, 554273.92it/s]


In [8]:
async def main():
    async with aiohttp.ClientSession() as session:
        pokemon_url = 'https://pokeapi.co/api/v2/pokemon/151'
        async with session.get(pokemon_url) as resp:
            pokemon = await resp.json()
            print(pokemon['name'])

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [2]:
headers = {'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
           'sec-ch-ua-mobile':'0',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }

def get_inn_vie_purch(purch):
    url="https://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber="+purch
    r = requests.get(url, headers = headers)
    text = r.content.decode('utf-8')
    soup = BeautifulSoup(text)
    winer_list = soup.find_all('table',{'class': 'blockInfo__table tableBlock'})
    if len(winer_list) > 3:
        soup = BeautifulSoup(str(winer_list[3]))
    elif len(winer_list) == 3:
        soup = BeautifulSoup(str(winer_list[2]))
    else:
        return
    contract_link = soup.find('a')
    try:
        contract_url = contract_link.get('href')
    except Exception:
        return
    contract_url = 'https://zakupki.gov.ru' + contract_url
    try:
        contract_page = requests.get(contract_url, headers = headers)
    except Exception:
        return
    contract_page_text = contract_page.content.decode('utf-8')
    soup = BeautifulSoup(contract_page_text)
    spans = soup.find_all('span')
    ind_cur = -1
    for span in spans:
        if span.text == 'ИНН:': 
            ind_cur = spans.index(span)+1
            break
    if ind_cur == -1:
        return 
    else:
        return spans[ind_cur].text

In [ ]:
def start_download(purch):
    out = purch
    scraping = threading.Thread(target=get_inn_vie_purch, args=(out,))
    scraping.start()
    scraping.join()
    while threading.active_count()>10:
        time.sleep(0.01)